In [6]:
import pandas as pd
import numpy as np
from pysus.online_data import SINAN, FTP_Inspect, parquets_to_dataframe
from pysus.preprocessing.decoders import decodifica_idade_SINAN
decodifica_idade_SINAN
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl


# Importando base de dados da Poliomielite

In [7]:
years = SINAN.get_available_years('Poliomielite')

* Concatenando os dataframes disponíves em cada ano

In [8]:
df = pd.DataFrame()
for y in years:
    fn = SINAN.download('Poliomielite', y)
    df2 = parquets_to_dataframe(fn)
    df = pd.concat([df, df2])

In [4]:
df.shape

(4172, 168)

* Colunas com datas e ID's vem codificadas. No código abaixo executamos a decodificação

In [9]:
for cname in df.columns:
    if cname.startswith('DT_'):
        df[cname] = pd.to_datetime(df[cname], errors='coerce')
    elif cname.startswith('ID_'):
        try:
            df[cname] = pd.to_numeric(df[cname])
        except ValueError:
            continue

In [ ]:
display(df)

* Renomeando os estados de acordo com sua sigla

In [10]:
def func(a):
    estados = {
    '11': 'RO', '12': 'AC', '13': 'AM', '14': 'RR', '15': 'PA',
    '16': 'AP', '17': 'TO', '21': 'MA', '22': 'PI', '23': 'CE',
    '24': 'RN', '25': 'PB', '26': 'PE', '27': 'AL', '28': 'SE',
    '29': 'BA', '31': 'MG', '32': 'ES', '33': 'RJ', '35': 'SP',
    '41': 'PR', '42': 'SC', '43': 'RS', '50': 'MS', '51': 'MT',
    '52': 'GO', '53': 'DF'
}
    return estados[a]
    
df['SG_UF'] = df['SG_UF'].apply(lambda x: func(x))

In [8]:
print(f'NaN na coluna dos estados ', df['SG_UF'].isnull().sum())
print('NaN na coluna das datas ', df['DT_NOTIFIC'].isnull().sum())

NaN na coluna dos estados  0
NaN na coluna das datas  0


In [11]:
df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'])
df['ANO'] = df['DT_NOTIFIC'].dt.year

* Nosso objetivo é obter os casos por ano em cada estado. Logo executamos o groupby

In [12]:
selecionado = df[['SG_UF', 'ANO']]
agrupados = selecionado.groupby(['SG_UF', 'ANO'],as_index=False).size()
df2 = pd.DataFrame(agrupados)
df2.rename(columns={'size' : 'CASOS'}, inplace=True)
df2

,SG_UF,ANO,CASOS
0,AC,2012,6
1,AC,2013,4
2,AC,2014,6
3,AC,2015,3
4,AC,2016,3
...,...,...,...
242,TO,2016,5
243,TO,2017,6
244,TO,2018,6
245,TO,2019,4


### O arquivo 'BR_UF_2022.shp' contem as informações geométricas de cada região 
 * Shape file obtido do link abaixo:

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15774-divisao-territorial-brasileira.html.

* Nomeando cada coordenada com seu respetivo estado

In [13]:
estados = [
    'AC', 'AM', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN', 'PB',
    'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC',
    'RS', 'MS', 'MT', 'GO', 'DF', 'RO', 'RR'
]
series_estados = pd.Series(estados)
estados_gdf = gpd.read_file("./BR_UF_2022.shp")
estados_gdf['SG_UF'] = estados_gdf


/home/joao/anaconda3/envs/L3/lib/python3.9/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


* Juntamos as informações de cada região com suas respectivas coordenadas

In [24]:
merged = pd.merge(df2, estados_gdf, on='SG_UF')

In [15]:
merged

,SG_UF,ANO,CASOS,geometry
0,AC,2012,6,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
1,AC,2013,4,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
2,AC,2014,6,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
3,AC,2015,3,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
4,AC,2016,3,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
...,...,...,...,...
242,TO,2016,5,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."
243,TO,2017,6,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."
244,TO,2018,6,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."
245,TO,2019,4,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."


* Transformando a coluna ano para o formato que o Kepler irá entender

In [25]:
merged['ANO'] = pd.to_datetime(merged['ANO'], format='%Y').dt.strftime('%Y-01-01T00:00:00')


In [26]:
merged

,SG_UF,ANO,CASOS,geometry
0,AC,2012-01-01T00:00:00,6,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
1,AC,2013-01-01T00:00:00,4,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
2,AC,2014-01-01T00:00:00,6,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
3,AC,2015-01-01T00:00:00,3,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
4,AC,2016-01-01T00:00:00,3,"POLYGON ((-68.79282 -10.99957, -68.79367 -10.9..."
...,...,...,...,...
242,TO,2016-01-01T00:00:00,5,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."
243,TO,2017-01-01T00:00:00,6,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."
244,TO,2018-01-01T00:00:00,6,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."
245,TO,2019-01-01T00:00:00,4,"POLYGON ((-48.24830 -13.19239, -48.24844 -13.1..."


### Criando o geodataframe

In [27]:

gdf = gpd.GeoDataFrame(merged, geometry="geometry")
print(gdf.head())

  SG_UF                  ANO  CASOS  \
0    AC  2012-01-01T00:00:00      6   
1    AC  2013-01-01T00:00:00      4   
2    AC  2014-01-01T00:00:00      6   
3    AC  2015-01-01T00:00:00      3   
4    AC  2016-01-01T00:00:00      3   

                                            geometry  
0  POLYGON ((-68.79282 -10.99957, -68.79367 -10.9...  
1  POLYGON ((-68.79282 -10.99957, -68.79367 -10.9...  
2  POLYGON ((-68.79282 -10.99957, -68.79367 -10.9...  
3  POLYGON ((-68.79282 -10.99957, -68.79367 -10.9...  
4  POLYGON ((-68.79282 -10.99957, -68.79367 -10.9...  


In [28]:
gdf.to_file('dataframe.geojson', driver='GeoJSON') 

/home/joao/anaconda3/envs/L3/lib/python3.9/site-packages/geopandas/io/file.py:174: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
